In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [2]:
df17=pd.read_csv('data_icaller_17k.csv')
df17['Spam']=df17.Spam.replace({1:2})
not_label=pd.read_csv('not_label.csv')
not_label['Spam']=not_label.Spam.replace({0:1})
data_icaller_25k=pd.concat([df17, not_label], axis=0)
df=data_icaller_25k


In [3]:
df.describe()

,call_to,call_in,call_to_miss,call_in_miss,duration_call_to,duration_call_in,avg_duration_call_to,avg_duration_call_in,avg_in_contact,in_hour,avg_success,Spam
count,25572.000000,25572.000000,25572.000000,25572.000000,25572.000000,25572.000000,25572.000000,25572.000000,25572.000000,25572.000000,25572.000000,25572.000000
mean,18.634444,14.430197,15.603081,9.184029,1046.532345,764.184002,25.628671,19.596430,0.402647,39.928868,0.490941,1.000000
std,63.044699,33.027303,74.374359,24.658062,3731.220579,2434.227175,53.191038,49.618314,0.426544,125.476911,0.308517,0.816513
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,13.000000,0.000000,3.499956,0.000000,0.000000,2.000000,0.252242,0.000000
50%,3.000000,1.000000,4.000000,1.000000,117.000000,13.000000,14.259457,4.799948,0.200276,9.000000,0.500000,1.000000
75%,19.000000,19.000000,16.000000,9.000000,741.000000,711.000000,29.937685,25.332489,0.876975,50.000000,0.680000,2.000000
max,2333.000000,1366.000000,2812.000000,1718.000000,124623.000000,143204.000000,3745.625437,2182.445439,1.000000,4704.000000,1.000000,2.000000


In [4]:
df.columns

Index(['call_to', 'call_in', 'call_to_miss', 'call_in_miss',
       'duration_call_to', 'duration_call_in', 'avg_duration_call_to',
       'avg_duration_call_in', 'avg_in_contact', 'in_hour', 'avg_success',
       'Spam'],
      dtype='object')

In [5]:
df=df.drop(df.loc[((df.Spam==2)&(df.call_to==0)&(df.call_in!=0)&(df.call_to_miss==0))|((df.Spam==1)&(df.avg_success==0)&(df.call_to==0)&(df.call_in==0))].index)
df.shape

(21842, 12)

In [6]:
X = df.copy()#.drop(columns=['phone'])#,'call_in','call_to_miss','duration_call_in','duration_call_to','call_in_miss','call_to'])
y = X.pop("Spam")
X.shape

(21842, 11)

In [7]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score 
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, auc, log_loss, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn import preprocessing

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.preprocessing import binarize

import eli5
from eli5.sklearn import PermutationImportance

Using TensorFlow backend.


In [8]:

# for feature in X.columns:
#     X[feature]=X[feature].apply(np.log1p)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)
df_test=pd.read_csv('data_icaller_test.csv')
X_test = df_test.copy()#.drop(columns=['Spam'])#,'call_in','call_to_miss','duration_call_in','duration_call_to','call_in_miss','call_to'])
X_test["Spam"]=1
y_test = X_test.pop("Spam")
X_test.shape
# y_train = tf.keras.utils.to_categorical(y_train, number_of_classes)
# y_test = tf.keras.utils.to_categorical(y_test, number_of_classes)

(100000, 11)

In [10]:
logreg = LogisticRegression(multi_class='ovr')
h2=LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_LR = logreg.predict(X_test)
y_p_LR=logreg.predict_proba(X_test)
# THRESHOLD=0.5
# y_pred_LR  = binarize(y_p_LR[:,1].reshape(-1, 1), THRESHOLD).ravel()
# Y_test_pred = pred_proba_df.applymap(lambda x: 1 if x>i else 0)
y_pred_train=logreg.predict(X_train)

accuracy_train_LR = round(metrics.accuracy_score(y_pred_train, y_train),3)
accuracy_LR = round(metrics.accuracy_score(y_pred_LR, y_test),3)
# f1_score_LR = round(metrics.f1_score(y_test,y_pred_LR),3)
# precision_LR = round(metrics.precision_score(y_test,y_pred_LR),3)
# recall_LR = round(metrics.recall_score(y_test,y_pred_LR),3)
                          
print("Accuracy train LR: ", accuracy_train_LR)
print("Accuracy ttest LR: ", accuracy_LR)
# print("F1 score LR: ", f1_score_LR)
# print("Precision LR: ", precision_LR)
# print("Recall LR: ", recall_LR)
# print("Sum Spam: ", y_pred_LR.sum())
# print(y_p_LR[998])
print("Sum Spam: ",(y_pred_LR==2).sum())
print("Sum Member: ",(y_pred_LR==0).sum())
# perm = PermutationImportance(logreg, random_state=1).fit(X_test, y_test)
# eli5.show_weights(perm, feature_names = X_test.columns.tolist())

Accuracy train LR:  0.845
Accuracy ttest LR:  0.732
Sum Spam:  15746
Sum Member:  11071


In [11]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15289 entries, 1628 to 3330
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   call_to               15289 non-null  int64  
 1   call_in               15289 non-null  int64  
 2   call_to_miss          15289 non-null  int64  
 3   call_in_miss          15289 non-null  int64  
 4   duration_call_to      15289 non-null  float64
 5   duration_call_in      15289 non-null  float64
 6   avg_duration_call_to  15289 non-null  float64
 7   avg_duration_call_in  15289 non-null  float64
 8   avg_in_contact        15289 non-null  float64
 9   in_hour               15289 non-null  int64  
 10  avg_success           15289 non-null  float64
dtypes: float64(6), int64(5)
memory usage: 1.4 MB


In [12]:
y_test.sum()

100000

In [13]:
random_forest = RandomForestClassifier(n_estimators=100)
h1=RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)

y_pred_rf = random_forest.predict(X_test)
y_p_rf=random_forest.predict_proba(X_test)
y_pred_train=random_forest.predict(X_train)

# y_pred_rf  = binarize(y_p_rf[:,1].reshape(-1, 1), THRESHOLD).ravel()


accuracy_train_RF = round(metrics.accuracy_score(y_pred_train, y_train),3)
accuracy_RF = round(metrics.accuracy_score( y_test,y_pred_rf),3)
# f1_score_RF = round(metrics.f1_score( y_test,y_pred_rf),3)
# precision_RF = round(metrics.precision_score(y_test,y_pred_rf),3)
# recall_RF = round(metrics.recall_score(y_test,y_pred_rf),3)

print("Accuracy train RF: ", accuracy_train_RF)
print("Accuracy RF : ", accuracy_RF)
# print("F1 score RF: ", f1_score_RF)
# print("Precision RF: ", precision_RF)
# print("Recall RF: ", recall_RF)
# print("Sum Spam: ", y_pred_rf.sum())
# print(y_p_rf[998])
print("Sum Spam: ",(y_pred_rf==2).sum())
print("Sum Member: ",(y_pred_rf==0).sum())
# perm = PermutationImportance(random_forest, random_state=1).fit(X_test, y_test)
# eli5.show_weights(perm, feature_names = X_test.columns.tolist())
# # print(y_pred )
# print(y_p)

Accuracy train RF:  0.995
Accuracy RF :  0.726
Sum Spam:  3846
Sum Member:  23567


In [23]:
knn = KNeighborsClassifier(n_neighbors = 10,  weights = 'distance')
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
# y_p_knn=knn.predict_proba(X_test)


# y_pred_knn  = binarize(y_p_knn[:,1].reshape(-1, 1), THRESHOLD).ravel()


accuracy_KNN = round(metrics.accuracy_score(y_test,y_pred_knn),3)
# f1_score_KNN = round(metrics.f1_score(y_test,y_pred_knn),3)
# precision_KNN = round(metrics.precision_score(y_test,y_pred_knn),3)
# recall_KNN = round(metrics.recall_score(y_test,y_pred_knn),3)
# y_pred_knn
print("Accuracy KNN: ", accuracy_KNN)
# print("F1 score KNN: ", f1_score_KNN)
# print("Precision KNN: ", precision_KNN)
# print("Recall KNN: ", recall_KNN)
# print("Sum Spam: ", y_pred_knn.sum())
# print("Sum Spam: ",(y_pred_KNN==2).sum())
# print("Sum Member: ",(y_pred_KNN==0).sum())
# perm = PermutationImportance(knn, random_state=1).fit(X_test, y_test)
# eli5.show_weights(perm, feature_names = X_test.columns.tolist())


Accuracy KNN:  0.657


NameError: name 'y_pred_KNN' is not defined

In [15]:
svc = LinearSVC()
svc.fit(X_train, y_train)
y_pred_svc = svc.predict(X_test)
y_pred_train=svc.predict(X_train)

# y_p_svc=svc.predict_proba(X_test)

# y_pred_svc  = binarize(y_p_svc[:,1].reshape(-1, 1), THRESHOLD).ravel()


accuracy_train_SVC = round(metrics.accuracy_score(y_train,y_pred_train),3)
accuracy_SVC = round(metrics.accuracy_score( y_test,y_pred_svc),3)
# f1_score_SVC = round(metrics.f1_score(y_test,y_pred_svc),3)
# precision_SVC = round(metrics.precision_score( y_test,y_pred_svc),3)
# recall_SVC = round(metrics.recall_score( y_test,y_pred_svc),3)

print("Accuracy train SVC: ", accuracy_train_SVC)
print("Accuracy SVC: ", accuracy_SVC)
# print("F1 score SVC: ", f1_score_SVC)
# print("Precision SVC: ", precision_SVC)
# print("Recall SVC: ", recall_SVC)
# print("Sum Spam: ", y_pred_svc.sum())

# perm = PermutationImportance(svc, random_state=1).fit(X_test, y_test)
# eli5.show_weights(perm, feature_names = X_test.columns.tolist())

# print(y_pred )
# print(y_p)

Accuracy train SVC:  0.672
Accuracy SVC:  0.671


In [16]:
gaussian = BernoulliNB()
gaussian.fit(X_train, y_train)

y_pred_NB = gaussian.predict(X_test)
# y_p_NB=gaussian.predict_proba(X_test)
# THRESHOLD=0.5
# y_pred_NB  = binarize(y_p_NB[:,1].reshape(-1, 1), THRESHOLD).ravel()


accuracy_NB = round(metrics.accuracy_score(y_test,y_pred_NB),3)
# f1_score_NB = round(metrics.f1_score(y_test,y_pred_NB),3)
# precision_NB = round(metrics.precision_score(y_test,y_pred_NB),3)
# recall_NB = round(metrics.recall_score( y_test, y_pred_NB),3)

print("Accuracy NB: ", accuracy_NB)
# print("F1 score NB: ", f1_score_NB)
# print("Precision NB: ", precision_NB)
# print("Recall NB: ", recall_NB)
# print("Sum Spam: ", y_pred_NB.sum())

# perm = PermutationImportance(knn, random_state=1).fit(X_test, y_test)
# eli5.show_weights(perm, feature_names = X_test.columns.tolist())



Accuracy NB:  0.606


In [17]:
X_train

,call_to,call_in,call_to_miss,call_in_miss,duration_call_to,duration_call_in,avg_duration_call_to,avg_duration_call_in,avg_in_contact,in_hour,avg_success
1628,1,2,0,0,12.000000,2.0,11.998800,0.999950,1.000000,1,0.333333
6596,21,43,11,23,583.000000,2169.0,18.218693,32.863587,0.602041,67,0.612245
17012,1,0,6,0,10.251818,0.0,1.427168,0.000000,0.000000,6,0.140609
2004,8,8,18,22,297.000000,112.0,11.423033,3.733321,0.000000,30,0.250000
5880,99,87,107,45,12604.000000,4102.0,61.184436,31.075734,0.899408,214,0.541420
...,...,...,...,...,...,...,...,...,...,...,...
1434,39,21,15,23,1742.000000,801.0,32.259200,18.204504,0.806122,60,0.591837
4792,1,1,0,3,11.000000,29.0,10.998900,7.249819,0.000000,5,0.400000
13665,5,0,8,0,111.730160,0.0,8.287364,0.000000,0.000000,10,0.328450
8490,38,23,28,10,7669.000000,2239.0,116.196794,67.848279,0.888889,71,0.545455


In [18]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [19]:
number_of_classes=3
# y_train = tf.keras.utils.to_categorical(y_train, number_of_classes)
# y_test = tf.keras.utils.to_categorical(y_test, number_of_classes)

In [20]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(50, activation='relu', input_shape=[X_train.shape[1]]),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(25, activation='relu'),
    layers.Dropout(0.3),
#     layers.BatchNormalization(),
#     layers.Dense(10, activation='relu'),
#     layers.Dropout(0.3),
    layers.BatchNormalization(),
    
    layers.Dense(3, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                600       
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
batch_normalization (BatchNo (None, 50)                200       
_________________________________________________________________
dense_1 (Dense)              (None, 25)                1275      
_________________________________________________________________
dropout_1 (Dropout)          (None, 25)                0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 25)                100       
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 7

In [21]:
tf.keras.optimizers.Adam(
    learning_rate=0.005, beta_1=0.9, beta_2=0.999, epsilon=1e-010, amsgrad=False,
    name='adam'
)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [22]:
model.fit(X_train, y_train, epochs=300, batch_size=256)

Epoch 1/300


ValueError: in user code:

    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:788 run_step  **
 

In [ ]:
_, accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))

In [ ]:
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

In [26]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)
y_pred_MLP=model.predict(X_test)
accuracy_MLP = round(metrics.accuracy_score(y_test,y_pred_MLP),3)
# f1_score_MLP = round(metrics.f1_score(y_test,y_pred_MLP),3)
# precision_MLP = round(metrics.precision_score(y_test,y_pred_MLP),3)
# recall_MLP = round(metrics.recall_score( y_test,y_pred_MLP),3)

print("Accuracy MLP: ", accuracy_MLP)
# print("F1 score MLP: ", f1_score_MLP)
# print("Precision MLP: ", precision_MLP)
# print("Recall MLP: ", recall_MLP)
# print("Sum Spam: ", y_pred_MLP.sum())
print("Sum Spam: ",(y_pred_MLP==2).sum())
print("Sum Member: ",(y_pred_MLP==0).sum())
# perm = PermutationImportance(knn, random_state=1).fit(X_test, y_test)
# eli5.show_weights(perm, feature_names = X_test.columns.tolist())

Accuracy MLP:  0.652
Sum Spam:  18273
Sum Member:  16564


In [ ]:
y_pred_MLP


In [ ]:
models = pd.DataFrame({
    'Model': ['LogisticRegression', 'KNN', 'LinearSVC', 
              'Random Forest', 'Naive Bayes','MultiLayer Perceptron'],
    'Score': [accuracy_LR, accuracy_KNN, accuracy_SVC, 
              accuracy_RF,accuracy_NB,accuracy_MLP]
              })
models.sort_values(by='Score', ascending=False)

In [ ]:
# from tensorflow.keras import layers
# import tensorflow_probability as tfp

In [ ]:
# # Define the prior weight distribution as Normal of mean=0 and stddev=1.
# # Note that, in this example, the we prior distribution is not trainable,
# # as we fix its parameters.
# def prior(kernel_size, bias_size, dtype=None):
#     n = kernel_size + bias_size
#     prior_model = keras.Sequential(
#         [
#             tfp.layers.DistributionLambda(
#                 lambda t: tfp.distributions.MultivariateNormalDiag(
#                     loc=tf.zeros(n), scale_diag=tf.ones(n)
#                 )
#             )
#         ]
#     )
#     return prior_model


# # Define variational posterior weight distribution as multivariate Gaussian.
# # Note that the learnable parameters for this distribution are the means,
# # variances, and covariances.
# def posterior(kernel_size, bias_size, dtype=None):
#     n = kernel_size + bias_size
#     posterior_model = keras.Sequential(
#         [
#             tfp.layers.VariableLayer(
#                 tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
#             ),
#             tfp.layers.MultivariateNormalTriL(n),
#         ]
#     )
#     return posterior_model

In [ ]:
# train_size=X_train.shape[1]
# bnn_model_small = keras.Sequential([
#     layers.Dense(12, activation='relu', input_shape=[X_train.shape[1]]),
#     tfp.layers.DenseVariational(
#             units=12,
#             make_prior_fn=prior,
#             make_posterior_fn=posterior,
#             activation="sigmoid"),
#     tfp.layers.DenseVariational(
#             units=10,
#             make_prior_fn=prior,
#             make_posterior_fn=posterior,
#             activation="sigmoid"),
    
    
#     layers.Dense(1,activation='sigmoid'),
# ])



In [ ]:
# bnn_model_small.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

In [ ]:
# bnn_model_small.fit(X_train, y_train, epochs=500, batch_size=256)

In [ ]:
# y_pred_MLP=bnn_model_small.predict(X_test)>0.5

# accuracy_MLP = round(metrics.accuracy_score(y_test,y_pred_MLP),3)
# f1_score_MLP = round(metrics.f1_score(y_test,y_pred_MLP),3)
# precision_MLP = round(metrics.precision_score(y_test,y_pred_MLP),3)
# recall_MLP = round(metrics.recall_score( y_test,y_pred_MLP),3)

# print("Accuracy MLP: ", accuracy_MLP)
# print("F1 score MLP: ", f1_score_MLP)
# print("Precision MLP: ", precision_MLP)
# print("Recall MLP: ", recall_MLP)
# print("Sum Spam: ", y_pred_MLP.sum())

# perm = PermutationImportance(knn, random_state=1).fit(X_test, y_test)
# eli5.show_weights(perm, feature_names = X_test.columns.tolist())

In [ ]:
y.sum()

In [ ]:
# from collections import Counter
# from sklearn.datasets import make_classification
# from imblearn.over_sampling import SVMSMOTE
# from matplotlib import pyplot
# from numpy import where
# oversample = SVMSMOTE()
# X, y = oversample.fit_resample(X, y)
